In [1]:
import pandas as pd
import numpy as np
import random
import math
import operator
from collections import defaultdict

In [2]:
data = pd.read_csv('Payments.csv')
bad_nodes_df = pd.read_csv('bad_sender.csv')
print(data.shape)
# print(bad_nodes.shape)
# print(data.loc[0])
graph_df = data[['Sender', 'Receiver']]
#print(graph_df.shape)
# print(graph_df.iloc[0])
graph_df.to_csv('graph.txt', sep=' ', index=False, header=False)
#type(bad_nodes)

(130535, 3)


In [3]:
# # Assuming df is your DataFrame and 'column1' and 'column2' are the names of the columns
# unique_values_column1 = graph_df['Sender'].unique()
# unique_values_column2 = graph_df['Receiver'].unique()

# # Concatenate unique values from both columns and convert them into a set to remove duplicates
# unique_values_set = set(unique_values_column1) | set(unique_values_column2)

# # Convert the set back into a list
# unique_values_list = list(unique_values_set)
# print(len(unique_values_list))

In [4]:
def get_nodes_from_graph_file(graph_file):
    graph_nodes = []
    edges_list = []
    with open (graph_file, 'r') as file:
        edges_list = file.readlines()
    for edge in edges_list:
        src, dst = edge.split(' ')
        src, dst = int(src), int(dst[:-1])
        #print(src,dst)
        if src not in graph_nodes:
            graph_nodes.append(src)
        if dst not in graph_nodes:
            graph_nodes.append(dst)
        
    return graph_nodes
            

In [5]:
nodes = get_nodes_from_graph_file('graph.txt')
#print(nodes)
#print(len(nodes))
#print(sorted(nodes))
#mapping

sorted_nodes_list = sorted(nodes)
mapping = {}
for i, node in enumerate(sorted_nodes_list):
    mapping[node] = i
#print(mapping)

In [6]:
def get_edges_from_graph_file(graph_file):
    edges = defaultdict(list)
    edges_list = []
    with open (graph_file, 'r') as file:
        edges_list = file.readlines()
    for edge in edges_list:
        src, dst = edge.split(' ')
        src, dst = int(src), int(dst[:-1])
        src = mapping[src]
        dst = mapping[dst]
        edges[src].append(dst)
        #print(mapping[src], mapping[dst])
    return edges

In [7]:
edges = get_edges_from_graph_file('graph.txt')
#mapping = mapping_nodes(nodes)
#print(edges)
#print(len(edges[1309]))
#print(len(edges))
# for key in edges.keys():
#     #key = mapping[key]
#     print(key)
#print(max(edges.keys()))
    

In [8]:
n = len(nodes)
#initialising the Transition matrix and vector
T = np.zeros((n,n))
#v = np.zeros(n)

for src in edges:
    length = len(edges[src])
    for dst in edges[src]:
        T[dst][src] = 1/length
# print(T)
# print(T.shape)

In [19]:
bad_Nodes = bad_nodes_df.values
bad_nodes_list = bad_Nodes.tolist()
# print(bad_nodes)
# print(type(bad_nodes))
bad_nodes = []
for i in bad_nodes_list:
    bad_nodes.append(mapping[i[0]])

print(bad_nodes)
# Initialize the vector with 0.5
v = np.full(n, 0.5) #n = number of nodes

v[bad_nodes] = 0

# Randomly select 20 indices to set their values to 1
good_nodes = np.random.choice(np.setdiff1d(np.arange(799), bad_nodes), 20, replace=False)
v[good_nodes] = 1
# v = good * [1] + bad * [0] + (len(nodes) - good - bad) * [0.5]
# random.shuffle(v) # random shuffling the good and the bad nodes
v = v/np.sum(v) # normalizing the vector to have sum of elements = 1
#print(v)

# for i in bad_nodes:
#     print(v[i], "\n")
print(v.sum())

[284, 248, 487, 144, 363, 30, 203, 41, 47, 246, 552, 157, 6, 33, 650, 97, 433, 640, 75, 696]
1.0000000000000002


In [25]:
b = 0.85 # value of damping factor
temp = v 
t = v
t = b * np.dot(T,t) + (1 - b) * v
i = 1 # number of iterations

while np.linalg.norm(temp - t) > 0: # iterate till the vector converges
    temp = t
    t = b * np.dot(T,t) + (1 - b) * v
    i = i+1

print("Number of iterations to converge: {}".format(i)) # print the number of iterations

trust_rank_node_score = {} # to store the node and the corresponding trustrank score
for k in range(1, len(t)):
    #if t[k] != 0:
        #trust_rank_node_score.append([k, t[k]])
        trust_rank_node_score[k] = t[k]
# sort the trustrank scores in the decreasing order
#trust_rank_node_score = sorted(trust_rank_node_score, key = operator.itemgetter(1), reverse = False)
print(T,"\n\n")
print(t,"\n\n")

Number of iterations to converge: 40
[[0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.0019685 ]
 [0.         0.00102145 0.         ... 0.         0.         0.        ]
 ...
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]] 


[2.54872860e-04 1.99891047e-04 2.19504612e-04 2.91768774e-04
 1.87748208e-04 2.51065119e-04 5.05438255e-04 1.97740948e-04
 1.87734668e-04 2.66464474e-04 2.11286686e-04 1.87734668e-04
 3.29472091e-04 3.54262024e-04 1.88829554e-04 2.61914153e-04
 1.88020133e-04 2.68323784e-04 1.88445338e-04 4.63393144e-04
 1.97902403e-04 1.99017712e-04 2.55598684e-04 1.88671022e-04
 2.09281480e-04 1.99300522e-04 2.35366427e-04 3.88895272e-04
 2.01931052e-04 1.88534217e-04 6.02873365e-07 1.88252113e-04
 4.10312675e-04 2.56722889e-

In [30]:
trust_rank_node_score[487]

0.0